In [ ]:
import pickle

with open('onehot_encoder.pkl', 'rb') as file:
    onehot_encoder = pickle.load(file)

In [ ]:
import pandas as pd
df1=pd.read_csv('/content/df1-synthetic_insurance_dataset_fixed.csv')
df1['Annual_Income'].min()

20123.03

In [ ]:
!pip install -q streamlit
!pip install -q localtunnel
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


# **Insurance Risk & Claim Dataset**

In [ ]:
%%writefile app1.py
import streamlit as st
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import pickle

# Load models and encoders
logi_model = joblib.load('/content/logi_model.pkl')
logis_model = joblib.load('/content/logistic_regression_model.pkl')
loaded_model = load_model('/content/my_model.h5')
with open('/content/onehot_encoder.pkl', 'rb') as file:
    onehot_encoder = pickle.load(file)

# Read dataset for scaling
df1 = pd.read_csv('/content/df1-synthetic_insurance_dataset_fixed.csv')

# Get min and max values for scaling
min_values = df1[['Annual_Income', 'Premium_Amount', 'Claim_Amount']].min()
max_values = df1[['Annual_Income', 'Premium_Amount', 'Claim_Amount']].max()

# Fit the scaler
min_max_values = np.array([min_values, max_values])
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(min_max_values)

# Streamlit app with tabs
st.title('Insurance Fraud Risk Prediction App')

# Create tabs
tab1, tab2, tab3 = st.tabs(['Claim Prediction', 'Fraud Detection', 'Advanced Fraud Risk Model'])

with tab1:
    st.header('Claim Prediction')
    customer_age = st.number_input('Customer Age', value=30, key='lr_age')
    annual_income_raw = st.number_input('Annual Income (Raw Value)', value=50000.0, key='lr_income')
    premium_amount_raw = st.number_input('Premium Amount (Raw Value)', value=1000.0, key='lr_premium')
    vehicle_property_age = st.number_input('Vehicle/Property Age', value=5, key='lr_vpa')
    claim_history = st.number_input('Claim History', value=1, key='lr_history')

    policy_type = st.selectbox('Policy Type', ['Auto', 'Health', 'Life', 'Property'], key='lr_policy')
    gender = st.selectbox('Gender', ['Female', 'Male', 'Other'], key='lr_gender')

    policy_type_encoded = [1 if policy_type == p else 0 for p in ['Auto', 'Health', 'Life', 'Property']]
    gender_encoded = [1 if gender == g else 0 for g in ['Female', 'Male', 'Other']]

    scaled_features = scaler.transform([[annual_income_raw, premium_amount_raw, 0]])  # Add a placeholder for Claim_Amount
    annual_income, premium_amount, _ = scaled_features[0]  # Ignore the third value

    new_data = pd.DataFrame({
        'Customer_Age': [customer_age],
        'Policy_Type_Auto': [policy_type_encoded[0]],
        'Policy_Type_Health': [policy_type_encoded[1]],
        'Policy_Type_Life': [policy_type_encoded[2]],
        'Policy_Type_Property': [policy_type_encoded[3]],
        'Gender_Female': [gender_encoded[0]],
        'Gender_Male': [gender_encoded[1]],
        'Gender_Other': [gender_encoded[2]],
        'Annual_Income': [annual_income],
        'Vehicle_Age_Property_Age': [vehicle_property_age],
        'Premium_Amount': [premium_amount],
        'Claim_History': [claim_history]
    })

    if st.button('Predicts Claim Prediction'):
        try:
            predictions = logis_model.predict_proba(new_data)
            predicted_class = logis_model.predict(new_data)
            risk_labels = {0: 'Filed Claim 2 or Below / did not file a claim Single time (0)', 1: 'Already Filed Claims More than 2 (1)'}
            risk_score = risk_labels.get(predicted_class[0], 'Unknown')
            st.write(f"Predicted class: {predicted_class[0]}")
            st.write(f"Risk Score: {risk_score}")
            st.write(f"Class probabilities: {predictions}")
        except Exception as e:
            st.error(f"An error occurred: {e}")

with tab2:
    st.header('Fraud Detection')
    vehicle_age_property_age = st.number_input('Vehicle/Property Age', value=0, key='simple_vpa')
    premium_amount_raw = st.number_input('Premium Amount (Raw Value)', key='simple_premium')
    claim_amount_raw = st.number_input('Claim Amount (Raw Value)', key='simple_claim')
    claim_history = st.number_input('Claim History', value=0, key='simple_history')

    scaled_features = scaler.transform([[0, premium_amount_raw, claim_amount_raw]])  # Placeholder for missing feature
    _, premium_amount, claim_amount = scaled_features[0]  # Ignore the first value


    new_data = pd.DataFrame({
        'Vehicle_Age_Property_Age': [vehicle_age_property_age],
        'Premium_Amount': [premium_amount],
        'Claim_Amount': [claim_amount],
        'Claim_History': [claim_history]
    })

    if st.button('Predict Simple Fraud'):
        try:
            predictions = logi_model.predict_proba(new_data)
            predicted_class = logi_model.predict(new_data)
            risk_labels = {0: 'Genuine (0)', 1: 'Fraud (1)'}
            risk_score = risk_labels.get(predicted_class[0], 'Unknown')
            st.write(f"Predicted class: {predicted_class[0]}")
            st.write(f"Risk Score: {risk_score}")
            st.write(f"Class probabilities: {predictions}")
        except Exception as e:
            st.error(f"An error occurred: {e}")

with tab3:
    st.header('Risk Classification')
    customer_age = st.number_input('Customer Age', value=58, key='adv_age')
    annual_income_raw = st.number_input('Annual Income (Raw Value)', key='adv_income')
    premium_amount_raw = st.number_input('Premium Amount (Raw Value)', key='adv_premium')
    claim_amount_raw = st.number_input('Claim Amount (Raw Value)', key='adv_claim')

    scaled_features = scaler.transform([[annual_income_raw, premium_amount_raw, claim_amount_raw]])
    annual_income, premium_amount, claim_amount = scaled_features[0]

    vehicle_property_age = st.number_input('Vehicle/Property Age', value=7, key='adv_vpa')
    claim_history = st.number_input('Claim History', value=1, key='adv_history')
    fraudulent_claim = st.number_input('Fraudulent Claim', value=0, key='adv_fraud')

    policy_type = st.selectbox('Policy Type', ['Auto', 'Health', 'Life', 'Property'], key='adv_policy')
    gender = st.selectbox('Gender', ['Female', 'Male', 'Other'], key='adv_gender')

    categorical_data = pd.DataFrame({'Policy_Type': [policy_type], 'Gender': [gender]})
    encoded_data = onehot_encoder.transform(categorical_data).reshape(1, -1)

    new_data = np.array([[customer_age, annual_income, vehicle_property_age, claim_history, premium_amount, claim_amount, fraudulent_claim]])
    final_input = np.concatenate([new_data, encoded_data], axis=1)

    if st.button('Predict Advanced Fraud Risk'):
        try:
            predictions = loaded_model.predict(final_input)
            predicted_class = np.argmax(predictions, axis=1)
            risk_labels = {0: 'Low (0)', 1: 'Medium (1)', 2: 'High (2)'}
            risk_score = risk_labels.get(predicted_class[0], 'Unknown')
            st.write(f"Predicted class: {predicted_class[0]}")
            st.write(f"Risk Score: {risk_score}")
            st.write(f"Class probabilities: {predictions}")
        except Exception as e:
            st.error(f"An error occurred: {e}")


Writing app1.py


In [ ]:
ipv4 = !curl ipv4.icanhazip.com
ipv4

['35.196.155.135']

In [ ]:
!streamlit run app1.py  &>/content/logs.txt & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://sour-actors-bake.loca.lt
^C
